### Imports

In [25]:
from ta import add_all_ta_features
import os
import holidays
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from ta.momentum import RSIIndicator
import plotly.graph_objects as go

from sklearn.model_selection import train_test_split


from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
import pandas_ta as ta
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import numpy as np

### Read data

In [27]:
folder_path = './data/'
files = os.listdir(folder_path)
print(files)

list_to_concat = []
for file in files:
    if 'txt' in file:
        list_to_concat.append(pd.read_csv(f'./data/{file}', sep=','))


['AFLT_daily_010114_010124.txt']


### Preprocessing

In [26]:
def column_name_preprocesing(df):
    '''
    change names of columns
    drop columns
    add change in price column
    '''
    df = df.copy()
    df.rename(columns={'<OPEN>': 'open', '<HIGH>': 'high', '<LOW>': 'low', '<CLOSE>': 'close', '<VOL>': 'volume', '<TICKER>':'symbol','<DATE>':'datetime'}, inplace=True)
    columns_to_keep = ['symbol', 'datetime', 'open', 'close', 'high', 'low', 'volume']
    df = df[columns_to_keep]
    df.sort_values(by = ['symbol','datetime'], inplace = True)
    df['change_in_price'] = df['close'].diff()
    return df

In [28]:
result = pd.concat(list_to_concat, ignore_index=True)
result = column_name_preprocesing(result)
result['datetime'] = pd.to_datetime(result['datetime'], format='%Y%m%d')
df = result
date_range = pd.date_range(start=df['datetime'].min(), end=df['datetime'].max(), freq='D')
df = pd.DataFrame(date_range, columns=['datetime']).merge(df, how='left', on='datetime')
df = df.fillna(method='ffill')
df['rsi14'] = ((RSIIndicator(close = df['close'], window = 14).rsi() - 50)/50)
df['rsi30'] = ((RSIIndicator(close = df['close'], window = 14).rsi() - 50)/50)

df = add_all_ta_features(
    df, open="open", high="high", low="low", close="close", volume="volume", fillna=True)
df['change_in_price'] = df['change_in_price'].shift(-1)
df['change_in_price'] = df['change_in_price'].apply(lambda x: 1 if x>=0 else 0)

C:\Users\Matvey\AppData\Local\Temp\ipykernel_12500\806409226.py:7: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill')


In [29]:
df.drop(columns=['datetime','symbol'], inplace=True)
df.dropna(inplace=True)

### Adaboost

In [30]:
X_Cols = df.drop(['change_in_price'], axis = 1)
Y_Cols = df['change_in_price']

X_train, X_test, y_train, y_test = train_test_split(X_Cols, Y_Cols, shuffle=False, test_size=0.2)

In [7]:
base_classifier = DecisionTreeClassifier(max_depth=1)
ada_boost = AdaBoostClassifier(base_classifier, n_estimators=50, random_state=42)
ada_boost.fit(X_train, y_train)
y_pred = ada_boost.predict(X_test)

print('Correct Prediction (%): ', accuracy_score(y_test, ada_boost.predict(X_test), normalize = True) * 100.0)

Correct Prediction (%):  51.30674002751031


In [22]:
from sklearn.model_selection import RandomizedSearchCV


In [32]:
my_list = np.logspace(-2, 0, num=100, endpoint=True)
my_list = [round(x, 2) for x in my_list]
my_list = list(set(my_list))

param_dist = {
    'base_estimator__max_depth': list(range(1, 21, 1)),
    'n_estimators': list(range(10, 300, 10)),
    'learning_rate': my_list
}

base_classifier = DecisionTreeClassifier()
ada_boost = AdaBoostClassifier(base_classifier)

ada_boost_random = RandomizedSearchCV(ada_boost, param_distributions=param_dist, n_iter=50, cv=3, random_state=42, n_jobs = -1)
ada_boost_random.fit(X_train, y_train)

print('Correct Prediction (%): ', accuracy_score(y_test, ada_boost_random.predict(X_test), normalize = True) * 100.0)

C:\Users\Matvey\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_search.py:930: FutureWarning: Parameter 'base_estimator' of AdaBoostClassifier is deprecated in favor of 'estimator'. See AdaBoostClassifier's docstring for more details.
  self.best_estimator_ = clone(base_estimator).set_params(


Correct Prediction (%):  49.38101788170564


In [33]:
ada_boost_random.best_params_

{'n_estimators': 160, 'learning_rate': 0.14, 'base_estimator__max_depth': 1}

In [34]:
base_classifier = DecisionTreeClassifier(max_depth=1)
ada_boost = AdaBoostClassifier(base_classifier, n_estimators=160, learning_rate=0.14)
ada_boost.fit(X_train, y_train)
print('Correct Prediction (%): ', accuracy_score(y_test, ada_boost.predict(X_test), normalize = True) * 100.0)

Correct Prediction (%):  49.38101788170564
